# Outline

- topic modelling by using **Latent Dirichlet Allocation**
- train data is `matichon.json` that contains about 16K articles of Thai news by Matichon between 2015-2018
- filter only `politics` category 

# Import

In [1]:
import pandas as pd
import re

from gensim.models import LdaModel  # LDA
from gensim.corpora.dictionary import Dictionary  # for count words 

import pythainlp
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

# Load data

- `headline` and `article` are untokenized text
- filter by `category` == `politics`

In [2]:
df = pd.read_json('data/matichon.json')
df

,headline,article,date,category,url,id
0,ซาอุฯจ่อเปิดไฟเขียวให้สิทธิหญิงม่าย-หย่าร้างปก...,(2 ธ.ค.58) หนังสือพิมพ์อัล ริยาดของทางการซาอุด...,2015-12-04 03:35:18,foreign,https://www.matichon.co.th/foreign/news_293,293
1,"""ไก่อู""ชี้ ตู่-เต้น ไม่ได้มีหน้าที่ตรวจสอบทุจร...","""บิ๊กป้อม"" แจง ครม. มีความพยายามยุยงปลุกปั่นให...",2015-12-04 04:10:49,politics,https://www.matichon.co.th/politics/news_329,329
2,"เปิดใจ ""โบว์ แวนดา"" ระหว่างรอยิ้มได้เต็มที่ในว...",แม้จะทำหน้าที่ภรรยาที่ดีมาเฝ้าปอ – ทฤษฎี สหวงษ...,2015-12-04 06:30:11,entertainment,https://www.matichon.co.th/entertainment/news_375,375
3,"""นาย ณภัทร"" ปลื้มคนชมแชมป์ขึ้นปกนิตยสารแห่งปี ...",กลายเป็นดาราหนุ่มเนื้อหอมแฟนคลับแน่น กระแสมาแร...,2015-12-04 07:10:26,entertainment,https://www.matichon.co.th/entertainment/news_393,393
4,"คอแทบหัก! แม่ยกแห่คล้องพวงมาลัยักษ์ ""บอย ศิริช...",แสดงดีจนเป็นที่ถูกอกถูกใจแฟนคลับ จนได้รับพวงมา...,2015-12-05 05:26:20,entertainment,https://www.matichon.co.th/entertainment/news_445,445
...,...,...,...,...,...,...
17104,โบว์ ณัฏฐา แจ้งความ พล.ต.อ.ศรีวราห์ ถูกพาดพิง...,เมื่อวันที่ 5 ก.ค. ที่ สน.พญาไท น.ส.ณัฏฐา มหัท...,2018-07-05 13:25:45,politics,https://www.matichon.co.th/politics/news_1029607,1029607
17105,ภาพบรรยากาศ ขุดทางระบายน้ำ เร่งนำ 13 ชีวิตออกจ...,วันที่ 5 กรกฎาคม เจ้าหน้าที่ขุดทางระบายน้ำที่ด...,2018-07-05 13:33:10,region,https://www.matichon.co.th/region/news_1029619,1029619
17106,สนช.ผ่านพ.ร.บ.สงฆ์ 3 วาระรวด พระมหากษัตริย์ทรง...,"สนช.ผ่าน พ.ร.บ.สงฆ์ 3 วาระรวด ""วิษณุ"" แจงสาระส...",2018-07-05 13:33:27,politics,https://www.matichon.co.th/politics/news_1029636,1029636
17107,นานาทรรศนะเพิ่มค่าปรับหมอ 5ล้านบ. สกัดเบี้ยว...,หมายเหตุ – จากกรณีที่ กระทรวงศึกษาธิการ (ศธ.) ...,2018-07-05 13:53:26,education,https://www.matichon.co.th/education/news_1029668,1029668


In [3]:
df = df[df['category']=='politics']
df

,headline,article,date,category,url,id
1,"""ไก่อู""ชี้ ตู่-เต้น ไม่ได้มีหน้าที่ตรวจสอบทุจร...","""บิ๊กป้อม"" แจง ครม. มีความพยายามยุยงปลุกปั่นให...",2015-12-04 04:10:49,politics,https://www.matichon.co.th/politics/news_329,329
16,"""เลขาธิการ คสช."" กำชับ กำลังพล เร่งทำความเข้าใ...",เมื่อวันที่ 12มกราคม ที่ กองบัญชาการกองทัพบก พ...,2016-01-12 19:42:20,politics,https://www.matichon.co.th/politics/news_981,981
19,ปชป.ค้านขุดคอคอดกระ ชี้ไม่ควรตัดประเทศเป็น2ส่วน,เมื่อวันที่ 12 มกราคมพิ นายนิฏฐ์ อินทรสมบัติ ร...,2016-01-13 10:20:24,politics,https://www.matichon.co.th/politics/news_1193,1193
38,"""บิ๊กหมู"" ยกคณะไป""อีสาน""เร่งติดตามโครงการ""แก้ม...",เมื่อเวลา 08.00 น. วันที่ 14 มกราคม ที่ฝูงเครื...,2016-01-14 09:42:20,politics,https://www.matichon.co.th/politics/news_2268,2268
40,'คำนูณ' จับตากรธ.บัญญัติคลื่นความถี่ ผลการต่อส...,นายคำนูณ สิทธิสมาน สมาชิกสภาขับเคลื่อนการปฎิรู...,2016-01-14 10:40:15,politics,https://www.matichon.co.th/politics/news_2292,2292
...,...,...,...,...,...,...
17099,เชื่อหรือไม่ ? 'พลังประชารัฐ' จะชนะเลือกตั้ง '...,จากกรณีนายภิรมย์ พลวิเศษ อดีตส.ส.นครราชสีมา พร...,2018-07-05 10:21:09,politics,https://www.matichon.co.th/politics/news_1029200,1029200
17100,"'วิญญัติ'จี้อัยการนำตัว""ตั๊น""-พวกฟ้องศาลเป็นจำ...",เมื่อวันที่ 5 กรกฎาคม นายวิญญัติ ชาติมนตรี เล...,2018-07-05 11:58:11,politics,https://www.matichon.co.th/politics/news_1029215,1029215
17103,พท. แฉอีก! วิธีดูดอดีตส.ส. ใช้ขรก.ชักจูง-อ้างช...,"""เพื่อไทย"" ขู่ ถ้าไม่หยุดกดดันคนโดนดูด อาจจะมี...",2018-07-05 13:08:24,politics,https://www.matichon.co.th/politics/news_1029549,1029549
17104,โบว์ ณัฏฐา แจ้งความ พล.ต.อ.ศรีวราห์ ถูกพาดพิง...,เมื่อวันที่ 5 ก.ค. ที่ สน.พญาไท น.ส.ณัฏฐา มหัท...,2018-07-05 13:25:45,politics,https://www.matichon.co.th/politics/news_1029607,1029607


# tokenize

In [4]:
## function for tokenize
## remove stopwords
STOPWORDS = thai_stopwords() 

def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [token.strip('\'').strip('\"') for token in tokens] # remove quotations before/after token
    tokens = [token for token in tokens if token not in STOPWORDS] # remove stopwords
    tokens = [token for token in tokens if re.match(r'[A-Za-zก-๙0-9][A-Za-zก-๙0-9\.\-]+', token)] # token pattern : only words/digits
    return tokens

tokenize("เปิดใจชัดๆ กับ'ชูวิทย์' สบู่ ขัน ต้องเตรียมไว้! ในคุกไม่มี 7-11")

['เปิดใจ', 'ชัด', 'ชู', 'วิทย์', 'สบู่', 'ขัน', 'เตรียม', 'คุก', '11']

In [5]:
df['tokens'] = df['article'].apply(tokenize)

/var/folders/0l/v6xrjrh90pl23576c_1w0hd40000gn/T/ipykernel_5537/480646713.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['article'].apply(tokenize)


# create corpus

In [6]:
## create vocab <> ID dictionary
dictionary = Dictionary(df['tokens']) 

## create bag of words of each document
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

In [7]:
## `dictionary` holds pairs of (index, vocab) 
pd.DataFrame(dictionary.items())

,0,1
0,0,13.00
1,1,22
2,2,2557
3,3,กม.
4,4,กรณี
...,...,...
24507,24507,ทรงไว้
24508,24508,ธรรมยุต
24509,24509,นัม
24510,24510,พุทธมามกะ


# model

In [8]:
num_topics = 6

## instantiation
lda = LdaModel(corpus, num_topics=num_topics)

# show keywords in each topic

- `lda.get_topic_terms(topic_no)` gives list of `(word_index, probability of being the topic)`
- must `dictionary[word_index]` to get the word

In [9]:
## show top 20 words of topic 1
topic_no = 1

topic_df = pd.DataFrame(columns=['word','probability'])
for index, prob in lda.get_topic_terms(topic_no, topn=20):
    topic_df.loc[len(topic_df)] = [dictionary[index], round(prob, 3)]

topic_df

,word,probability
0,ประชาชน,0.011
1,ประเทศ,0.010
2,เรื่อง,0.009
3,ธ.,0.008
4,รัฐธรรมนูญ,0.007
5,สนช,0.007
6,กฎหมาย,0.006
7,วันที่,0.006
8,คน,0.006
9,ร่าง,0.006


In [10]:
## show top 20 words of all topics
for topic_no in range(num_topics):
    print(f"---------- TOPIC {topic_no} ----------")
    top20 = [dictionary[index] for index, prob in lda.get_topic_terms(topic_no, topn=20)]
    print(' | '.join(top20), '\n')

---------- TOPIC 0 ----------
คน | วันที่ | ปี | เรื่อง | บาท | ประชาชน | ไทย | พระ | ประเทศ | อ. | พล | ทำ | คดี | เจ้าหน้าที่ | ดำเนินการ | ล้าน | กรณี | ตรวจสอบ | เวลา | กล่าวว่า 

---------- TOPIC 1 ----------
ประชาชน | ประเทศ | เรื่อง | ธ. | รัฐธรรมนูญ | สนช | กฎหมาย | วันที่ | คน | ร่าง | ไทย | รัฐบาล | การเลือกตั้ง | คสช. | อ. | พล | กล่าวว่า | คณะกรรมการ | ปี | พิจารณา 

---------- TOPIC 2 ----------
ทำ | คน | ประชาชน | ประเทศ | เรื่อง | รัฐบาล | ผม | สร้าง | พล | ไทย | อ. | คสช. | ปี | แห่งชาติ | ร่างรัฐธรรมนูญ | กล่าวว่า | สังคม | วันที่ | คณะกรรมการ | เวลา 

---------- TOPIC 3 ----------
พล | อ. | วันที่ | คน | ประชาชน | พรรค | ทำ | คสช. | รัฐบาล | เรื่อง | ไทย | ปี | การเลือกตั้ง | ประวิตร | เวลา | พรรคการเมือง | กรณี | ประเทศ | ส.ส. | รัฐธรรมนูญ 

---------- TOPIC 4 ----------
เรื่อง | การเลือกตั้ง | พรรค | คน | พรรคการเมือง | วันที่ | คสช. | ทำ | อ. | กกต. | พล | ประชาชน | สนช | เวลา | กรณี | กฎหมาย | รัฐบาล | เลือกตั้ง | ผม | คณะกรรมการ 

---------- TOPIC 5 ----------
อ.

# apply trained model to new data

In [27]:
new_texts = ['นายกเดินทางไปญี่ปุ่น', 'ประชาชนรักประเทศ']
new_tokens = [tokenize(text) for text in new_texts]
new_doc_bow = [dictionary.doc2bow(tokens) for tokens in new_tokens]

In [29]:
## show probability of each topic
result = lda.get_document_topics(new_doc_bow)
for i, probs in enumerate(result):
    print(f'-- document {i} ---')
    for topic, prob in probs:
        print(f'topic {topic} : {prob:.3f}')
    print()

-- document 0 ---
topic 0 : 0.042
topic 1 : 0.042
topic 2 : 0.042
topic 3 : 0.042
topic 4 : 0.042
topic 5 : 0.790

-- document 1 ---
topic 0 : 0.042
topic 1 : 0.790
topic 2 : 0.042
topic 3 : 0.042
topic 4 : 0.042
topic 5 : 0.042

